# Single-View Geometry (Python)

## Usage
This code snippet provides an overall code structure and some interactive plot interfaces for the *Single-View Geometry* section of Assignment 3. In [main function](#Main-function), we outline the required functionalities step by step. Some of the functions which involves interactive plots are already provided, but [the rest](#Your-implementation) are left for you to implement.

## Package installation
- You will need [GUI backend](https://matplotlib.org/faq/usage_faq.html#what-is-a-backend) to enable interactive plots in `matplotlib`.
- In this code, we use `tkinter` package. Installation instruction can be found [here](https://anaconda.org/anaconda/tk).

# Common imports

In [2]:
% matplotlib tk
import matplotlib.pyplot as plt
import numpy as np
import numpy.linalg as nl

from PIL import Image

# Provided functions

In [3]:
def get_input_lines(im, min_lines=3):
    """
    Allows user to input line segments; computes centers and directions.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        min_lines: minimum number of lines required
    Returns:
        n: number of lines from input
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        centers: np.ndarray of shape (3, n)
            where each column denotes the homogeneous coordinates of the centers
    """
    n = 0
    lines = np.zeros((3, 0))
    centers = np.zeros((3, 0))

    plt.figure()
    plt.imshow(im)
    print('Set at least %d lines to compute vanishing point' % min_lines)
    while True:
        print('Click the two endpoints, use the right key to undo, and use the middle key to stop input')
        clicked = plt.ginput(2, timeout=0, show_clicks=True)
        if not clicked or len(clicked) < 2:
            if n < min_lines:
                print('Need at least %d lines, you have %d now' % (min_lines, n))
                continue
            else:
                # Stop getting lines if number of lines is enough
                break

        # Unpack user inputs and save as homogeneous coordinates
        pt1 = np.array([clicked[0][0], clicked[0][1], 1])
        pt2 = np.array([clicked[1][0], clicked[1][1], 1])
        # Get line equation using cross product
        # Line equation: line[0] * x + line[1] * y + line[2] = 0
        line = np.cross(pt1, pt2)
        lines = np.append(lines, line.reshape((3, 1)), axis=1)
        # Get center coordinate of the line segment
        center = (pt1 + pt2) / 2
        centers = np.append(centers, center.reshape((3, 1)), axis=1)

        # Plot line segment
        plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], color='b')

        n += 1

    return n, lines, centers

In [4]:
def plot_lines_and_vp(im, lines, vp):
    """
    Plots user-input lines and the calculated vanishing point.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        vp: np.ndarray of shape (3, )
    """
    bx1 = min(1, vp[0] / vp[2]) - 10
    bx2 = max(im.shape[1], vp[0] / vp[2]) + 10
    by1 = min(1, vp[1] / vp[2]) - 10
    by2 = max(im.shape[0], vp[1] / vp[2]) + 10

    plt.figure()
    plt.imshow(im)
    for i in range(lines.shape[1]):
        if lines[0, i] < lines[1, i]:
            pt1 = np.cross(np.array([1, 0, -bx1]), lines[:, i])
            pt2 = np.cross(np.array([1, 0, -bx2]), lines[:, i])
        else:
            pt1 = np.cross(np.array([0, 1, -by1]), lines[:, i])
            pt2 = np.cross(np.array([0, 1, -by2]), lines[:, i])
        pt1 = pt1 / pt1[2]
        pt2 = pt2 / pt2[2]
        plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], 'g')

    plt.plot(vp[0] / vp[2], vp[1] / vp[2], 'ro')
    plt.show()

In [5]:
def get_top_and_bottom_coordinates(im, obj):
    """
    For a specific object, prompts user to record the top coordinate and the bottom coordinate in the image.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        obj: string, object name
    Returns:
        coord: np.ndarray of shape (3, 2)
            where coord[:, 0] is the homogeneous coordinate of the top of the object and coord[:, 1] is the homogeneous
            coordinate of the bottom
    """
    plt.figure()
    plt.imshow(im)

    print('Click on the top coordinate of %s' % obj)
    clicked = plt.ginput(1, timeout=0, show_clicks=True)
    x1, y1 = clicked[0]
    # Uncomment this line to enable a vertical line to help align the two coordinates
    # plt.plot([x1, x1], [0, im.shape[0]], 'b')
    print('Click on the bottom coordinate of %s' % obj)
    clicked = plt.ginput(1, timeout=0, show_clicks=True)
    x2, y2 = clicked[0]

    plt.plot([x1, x2], [y1, y2], 'b')

    return np.array([[x1, x2], [y1, y2], [1, 1]])

In [6]:
# get_input_lines(np.random.rand(500,500))

# Your implementation

In [7]:
def get_vanishing_point(lines):
    num_lines = len(lines)
    itsec = np.array([0.0, 0.0, 0.0])
    for i in range(num_lines):
        for j in range(i):
            if i == j:
                continue
            cur_pred = np.cross(lines[i], lines[j])
            cur_pred /= cur_pred[-1]
            itsec += cur_pred
    itsec /= num_lines * (num_lines-1) / 2

    return itsec / itsec[-1]    

In [8]:
def get_horizon_line(vpts):
    p1, p2, p3 = vpts[:,0], vpts[:,1], vpts[:,2]
    if(abs(p1[1]) > 2000):
        # print('2,3:',p2,p3,np.cross(p2, p3))
        return np.cross(p2, p3)
    if(abs(p2[1]) > 2000):
        # print('1,3:',p2,p3,np.cross(p1, p3))
        return np.cross(p1, p3)
    if(abs(p3[1]) > 2000):
        # print('1,2:',p2,p3,np.cross(p1, p2))
        return np.cross(p1, p2)
    return np.ones(3)

In [9]:
def plot_horizon_line(im, line):
    a, b, c = line[0], line[1], line[2]
    plt.figure()
    plt.imshow(im)
#     print('hi', line)
#     print([int(-c/a), int(-(c+1023*b)/a)])
    plt.plot([0, 1023], [(int(-c/b)), (int(-(c+1023*a)/b))], 'b')
    plt.show()

In [10]:
def get_camera_parameters(vpts):
    [xi, yi, _], [xj, yj, _], [xk, yk, _] = vpts[:,0]/vpts[:,0][-1], vpts[:,1]/vpts[:,1][-1],vpts[:,2]/ vpts[:,2][-1]
    c11, c12, c13 = xi*xj + yi*yj, xi + xj, yi + yj
    c21, c22, c23 = xi*xk + yi*yk, xi + xk, yi + yk
    c31, c32, c33 = xk*xj + yk*yj, xk + xj, yk + yj
    
    eq = np.append((np.array([c11, c12, c13]) - np.array([c21, c22, c23])).reshape(1,3),
             (np.array([c21, c22, c23]) - np.array([c31, c32, c33])).reshape(1,3), axis=0)
        
    A, b = eq[:,1:], eq[:,0]
    u, v = nl.solve(A, b)
    f = np.sqrt(c12*u + c13*v - c11 - u*u - v*v)
    
    print("f, u, v: ",f, u, v)
    return f, u, v

In [11]:
def get_rotation_matrix(K, vpts):
    R = np.zeros((3,3))
    p1, p2, p3 = vpts[:,0], vpts[:,1], vpts[:,2]
    p1, p2, p3 = p1/p1[-1], p2/p2[-1], p3/p3[-1]
    
    xs = np.array([p1[0], p2[0], p3[0]])
    ys = np.array([p1[1], p2[1], p3[1]])
    
    arg_x, arg_y, arg_z = np.argmax(xs), np.argmax(np.abs(ys)), np.argmin(xs)
    assert(arg_x != arg_y and arg_x != arg_z and arg_z != arg_y)
    
    sorted_vpts = vpts[:,np.array([arg_x, arg_y, arg_z])]
    # print("Sorted: ",sorted_vpts)
    R = nl.inv(K) @ sorted_vpts
    R = R / nl.norm(R, axis=0)

    return R, sorted_vpts

In [12]:
def estimate_height(vpt, ref, target, horizon_line):
    # ref, tar are of [3,2], top and bottom homo points.
    rtop, rbot = ref[:, 0], ref[:, 1]
    ttop, tbot = target[:, 0], target[:, 1]
    
    bb_line = np.cross(rbot, tbot)
    v = np.cross(bb_line, horizon_line)
    v = v / v[-1]
    
    t = np.cross(np.cross(v, rtop), np.cross(ttop, tbot))
    t = t / t[-1]
    
    h_over_r = nl.norm(t[:2] - tbot[:2]) / nl.norm(ttop[:2] - tbot[:2])
    print("H/R", h_over_r)
    
    len_r = 66 # inches
    len_t = len_r / h_over_r
    
    return len_t

# Main function

In [13]:
im = np.asarray(Image.open('mypic.jpg'))
# Part 1
# Get vanishing points for each of the directions
num_vpts = 3
vpts = np.zeros((3, num_vpts))
for i in range(num_vpts):
    print('Getting vanishing point %d' % i)
    # Get at least three lines from user input
    n, lines, centers = get_input_lines(im)
    # <YOUR IMPLEMENTATION> Solve for vanishing point
    vpts[:, i] = get_vanishing_point(lines.T)
    # Plot the lines and the vanishing point
    plot_lines_and_vp(im, lines, vpts[:, i])
    
print("vpts: ", vpts)
# <YOUR IMPLEMENTATION> Get the ground horizon line
horizon_line = get_horizon_line(vpts)
horizon_line /= np.sqrt(horizon_line[0] ** 2 + horizon_line[1] ** 2)
# <YOUR IMPLEMENTATION> Plot the ground horizon line
print('Horizon line is: ',horizon_line)
plot_horizon_line(im, horizon_line)

# Part 2
# <YOUR IMPLEMENTATION> Solve for the camera parameters (f, u, v)
f, u, v = get_camera_parameters(vpts)
K = np.eye(3)
K[0][0], K[1][1], K[0][2], K[1][2] = 1/f, 1/f, u, v

# Part 3
# <YOUR IMPLEMENTATION> Solve for the rotation matrix
R, vpts = get_rotation_matrix(K, vpts)
print("R is: ",R)
# vpts here are sorted, X right most, Y vertcal, Z left most

# Part 4
# Record image coordinates for each object and store in map
objects = ('person', 'CSL building', 'the spike statue', 'the lamp posts')
coords = dict()
for obj in objects:
    coords[obj] = get_top_and_bottom_coordinates(im, obj)

# <YOUR IMPLEMENTATION> Estimate heights
for obj in objects[1:]:
    print('Estimating height of %s' % obj)
    height = estimate_height(vpts[:,1], coords[objects[0]], coords[obj], horizon_line)
    print(obj + " is " + str(int(height)) + " inches, which is " + str(int(height / 12)) + "\'" + str(int(height % 12)))


Getting vanishing point 0
Set at least 3 lines to compute vanishing point
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Getting vanishing point 1
Set at least 3 lines to compute vanishing point
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Getting vanishing point 2
Set at least 3 lines to compute vanishing point
Click the two endpoints, use the right key to undo, and use the mi

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in sqrt
  if sys.path[0] == '':


Click on the bottom coordinate of person
Click on the top coordinate of CSL building
Click on the bottom coordinate of CSL building
Click on the top coordinate of the spike statue
Click on the bottom coordinate of the spike statue
Click on the top coordinate of the lamp posts
Click on the bottom coordinate of the lamp posts
Estimating height of CSL building
H/R 0.011490524149493257
CSL building is 5743 inches, which is 478'7
Estimating height of the spike statue
H/R 0.013222617631819892
the spike statue is 4991 inches, which is 415'11
Estimating height of the lamp posts
H/R 0.17906012277636768
the lamp posts is 368 inches, which is 30'8
